# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:

* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM ERD

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [1]:
# Your code here
import pandas as pd
import sqlite3

conn = sqlite3.connect('data.sqlite')

In [9]:
q = """
SELECT *
FROM offices
LIMIT 1
"""
pd.read_sql(q, conn)

,officeCode,city,phone,addressLine1,addressLine2,state,country,postalCode,territory
0,1,San Francisco,+1 650 219 4782,100 Market Street,Suite 300,CA,USA,94080,NA


In [11]:
q = """
SELECT *
FROM employees
LIMIT 1
"""
pd.read_sql(q, conn)

,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle
0,1002,Murphy,Diane,x5800,dmurphy@classicmodelcars.com,1,,President


## Select the names of all employees in Boston 

Hint: join the employees and offices tables. Select the first and last name.

In [37]:
# Your code here
q = """
SELECT *
FROM employees
    JOIN offices
    USING(officeCode)
WHERE
    city == 'Boston'
"""
pd.read_sql(q, conn)

,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle,city,phone,addressLine1,addressLine2,state,country,postalCode,territory
0,1188,Firrelli,Julie,x2173,jfirrelli@classicmodelcars.com,2,1143,Sales Rep,Boston,+1 215 837 0825,1550 Court Place,Suite 102,MA,USA,02107,NA
1,1216,Patterson,Steve,x4334,spatterson@classicmodelcars.com,2,1143,Sales Rep,Boston,+1 215 837 0825,1550 Court Place,Suite 102,MA,USA,02107,NA


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by. Select the office code, city, and number of employees.

In [54]:
# Your code here
q = """
SELECT officeCode, city, COUNT(employeeNumber) as employeeCount
FROM employees
    RIGHT JOIN offices
    USING(officeCode)
GROUP BY offices.officeCode
ORDER BY employeeCount
"""
pd.read_sql(q, conn)

,officeCode,city,employeeCount
0,27,Boston,0
1,2,Boston,2
2,3,NYC,2
3,5,Tokyo,2
4,7,London,2
5,6,Sydney,4
6,4,Paris,5
7,1,San Francisco,6


## Write 3 questions of your own and answer them

In [68]:
"Which cutomer has spent the most?"

q = """
SELECT customerNumber, customerName, contactLastName, contactFirstName, phone, addressLine1, addressLine2, city, state, postalCode, country, 
       SUM(amount) AS totalSpent
FROM customers
    INNER JOIN payments
    USING(customerNumber)
GROUP BY customerNumber
ORDER BY totalSpent DESC
LIMIT 1
"""
pd.read_sql(q, conn)

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,totalSpent
0,141,Euro+ Shopping Channel,Freyre,Diego,(91) 555 94 44,"C/ Moralzarzal, 86",,Madrid,,28034,Spain,715738.98


In [74]:
"What are the top 5 spending countries?"

q = """
SELECT country, SUM(amount) AS totalSpent
FROM customers
    INNER JOIN payments
    USING(customerNumber)
GROUP BY country
ORDER BY totalSpent DESC
LIMIT 5
"""
pd.read_sql(q, conn)

,country,totalSpent
0,USA,3040029.52
1,Spain,994438.53
2,France,965750.58
3,Australia,509385.82
4,New Zealand,392486.59


In [78]:
"What are the top 3 products sold in terms of volume?"

q = """
SELECT productCode, productName, SUM(quantityOrdered) as volumeSold
FROM orderDetails
    INNER JOIN products
    USING(productCode)
GROUP BY productCode
ORDER BY volumeSold DESC
LIMIT 3
"""
pd.read_sql(q, conn)

,productCode,productName,volumeSold
0,S18_3232,1992 Ferrari 360 Spider red,1808
1,S18_1342,1937 Lincoln Berline,1111
2,S700_4002,American Airlines: MD-11S,1085


## Level Up 1: Display the names of every individual product that each employee has sold

Hint: You will need to use multiple `JOIN` clauses to connect all the way from employee names to product names.

In [130]:
# Your code here
q = """
SELECT employees.firstName, employees.lastName, 
        products.productName
FROM employees
    INNER JOIN customers
        on employees.employeeNumber = customers.salesRepEmployeeNumber
    INNER JOIN orders
        USING(customerNumber)
    INNER JOIN orderdetails
        USING(orderNumber)
    INNER JOIN products
        USING(productCode)
    GROUP BY productName
"""
pd.read_sql(q, conn)

,firstName,lastName,productName
0,Leslie,Jennings,18th Century Vintage Horse Carriage
1,Leslie,Jennings,18th century schooner
2,Leslie,Jennings,1900s Vintage Bi-Plane
3,Leslie,Jennings,1900s Vintage Tri-Plane
4,Leslie,Jennings,1903 Ford Model A
...,...,...,...
104,Leslie,Jennings,The Mayflower
105,Leslie,Jennings,The Queen Mary
106,Leslie,Jennings,The Schooner Bluenose
107,Leslie,Jennings,The Titanic


## Level Up 2: Display the number of products each employee has sold

Alphabetize the results by employee last name.

Hint: Use the `quantityOrdered` column from `orderDetails`. Also, think about how to group the data when some employees might have the same first or last name.

In [142]:
# Your code here
q = """
SELECT employees.firstName, employees.lastName, SUM(quantityOrdered) as products_sold
FROM employees
    INNER JOIN customers
        on employees.employeeNumber = customers.salesRepEmployeeNumber
    INNER JOIN orders
        USING(customerNumber)
    INNER JOIN orderdetails
        USING(orderNumber)
    INNER JOIN products
        USING(productCode)
    GROUP BY employees.employeeNumber
    ORDER BY employees.lastName ASC
"""
pd.read_sql(q, conn)

,firstName,lastName,products_sold
0,Loui,Bondur,6186
1,Larry,Bott,8205
2,Pamela,Castillo,9290
3,Julie,Firrelli,4227
4,Andy,Fixter,6246
5,Martin,Gerard,4180
6,Gerard,Hernandez,14231
7,Leslie,Jennings,11854
8,Barry,Jones,7486
9,Peter,Marsh,6632


## Level Up 3: Display the names employees who have sold more than 200 different products

Hint: this is different from the previous question because the quantity sold doesn't matter, only the number of different products

In [159]:
# Your code here
q = """
SELECT employees.firstName, employees.lastName, 
        COUNT(DISTINCT(productCode)) AS unique_products_sold
FROM employees
    INNER JOIN customers
        on employees.employeeNumber = customers.salesRepEmployeeNumber
    INNER JOIN orders
        USING(customerNumber)
    INNER JOIN orderdetails
        USING(orderNumber)
    INNER JOIN products
        USING(productCode)
    GROUP BY employees.employeeNumber
    HAVING unique_products_sold > 200
"""
pd.read_sql(q, conn)

,firstName,lastName,unique_products_sold


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!